## FEMA Disasters/ FEMA code

by Billy Martinez

This application reads data from the FEMA disaster file. Then extracts necessary columns, transforms values as needed and loads into FEMA table. Finally via SQL code, it loads common fields into NATURAL_DISASTER table for easy comparison with other source data. Original source: https://www.fema.gov/media-library/assets/documents/28318

In [36]:
import pandas as pd
from sqlalchemy import create_engine

In [37]:
csv_file = "FEMA_DECLARE.csv"
fema_df = pd.read_csv(csv_file, encoding='utf-8')
fema_df.head()

,ID,Disaster_Number,IH_Program_Declared,IA_Program_Declared,PA_Program_Declared,HM_Program_Declared,State,Declaration_Date,Disaster_Type,Incident_Type,Title,Incident_Begin_Date,Incident_End_Date,Disaster_Close_Out_Date,Place_Code,Declared_County_Area,Declaration_Rquest_Number
0,1,4419,Yes,No,Yes,Yes,AL,3/5/19,DR,Tornado,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",3/3/19,3/3/19,NaN,99081.0,Lee (County),19006
1,2,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99009.0,Clallam (County),19005
2,3,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99027.0,Grays Harbor (County),19005
3,4,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99029.0,Island (County),19005
4,5,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99031.0,Jefferson (County),19005


In [38]:
fema_df.columns

Index(['ID', 'Disaster_Number', 'IH_Program_Declared', 'IA_Program_Declared',
       'PA_Program_Declared', 'HM_Program_Declared', 'State',
       'Declaration_Date', 'Disaster_Type', 'Incident_Type', 'Title',
       'Incident_Begin_Date', 'Incident_End_Date', 'Disaster_Close_Out_Date',
       'Place_Code', 'Declared_County_Area', 'Declaration_Rquest_Number'],
      dtype='object')

In [39]:
new_fema_df = fema_df

In [40]:
new_fema_df['data_source'] = 'FEMA'

In [41]:
new_fema_df.columns

Index(['ID', 'Disaster_Number', 'IH_Program_Declared', 'IA_Program_Declared',
       'PA_Program_Declared', 'HM_Program_Declared', 'State',
       'Declaration_Date', 'Disaster_Type', 'Incident_Type', 'Title',
       'Incident_Begin_Date', 'Incident_End_Date', 'Disaster_Close_Out_Date',
       'Place_Code', 'Declared_County_Area', 'Declaration_Rquest_Number',
       'data_source'],
      dtype='object')

In [44]:
new_fema_df.head()

,id,disaster_number,ih_program_declared,ia_program_declared,pa_program_declared,hm_program_declared,state,declaration_date,disaster_type,incident_type,title,incident_begin_date,incident_end_date,disaster_close_out_date,place_code,declared_county_area,declaration_rquest_number,data_source
0,1,4419,Yes,No,Yes,Yes,AL,3/5/19,DR,Tornado,"SEVERE STORMS, STRAIGHT-LINE WINDS, AND TORNADOES",3/3/19,3/3/19,NaN,99081.0,Lee (County),19006,FEMA
1,2,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99009.0,Clallam (County),19005,FEMA
2,3,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99027.0,Grays Harbor (County),19005,FEMA
3,4,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99029.0,Island (County),19005,FEMA
4,5,4418,No,No,Yes,Yes,WA,3/4/19,DR,Severe Storm(s),"SEVERE WINTER STORMS, STRAIGHT-LINE WINDS, FLO...",12/10/18,12/24/18,NaN,99031.0,Jefferson (County),19005,FEMA


In [45]:
cols = []
for col in new_fema_df.columns:
    cols.append(col.lower())
new_fema_df.columns = cols
cols

['id',
 'disaster_number',
 'ih_program_declared',
 'ia_program_declared',
 'pa_program_declared',
 'hm_program_declared',
 'state',
 'declaration_date',
 'disaster_type',
 'incident_type',
 'title',
 'incident_begin_date',
 'incident_end_date',
 'disaster_close_out_date',
 'place_code',
 'declared_county_area',
 'declaration_rquest_number',
 'data_source']

In [47]:
myTable_df = new_fema_df[['id',
 'data_source',
 'disaster_number',
 'ih_program_declared',
 'ia_program_declared',
 'pa_program_declared',
 'hm_program_declared',
 'state',
 'declaration_date',
 'disaster_type',
 'incident_type',
 'title',
 'incident_begin_date',
 'incident_end_date',
 'disaster_close_out_date',
 'place_code',
 'declared_county_area',
 'declaration_rquest_number'
 ]]

In [22]:
myTable_df['disaster number'].head()

0    4419
1    4418
2    4418
3    4418
4    4418
Name: disaster number, dtype: int64

In [48]:
myTable_df.columns

Index(['id', 'data_source', 'disaster_number', 'ih_program_declared',
       'ia_program_declared', 'pa_program_declared', 'hm_program_declared',
       'state', 'declaration_date', 'disaster_type', 'incident_type', 'title',
       'incident_begin_date', 'incident_end_date', 'disaster_close_out_date',
       'place_code', 'declared_county_area', 'declaration_rquest_number'],
      dtype='object')

In [49]:
rds_connection_string = "postgres:my74ETERNITY@localhost:5432/fema_disasters_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [50]:
engine.table_names()

['location',
 'earthquake',
 'data_source',
 'wild_fire',
 'natural_disaster',
 'tornado',
 'fema']

In [53]:
myTable_df.to_sql(name='fema', con=engine, if_exists='append', index=False)